In [1]:
# !nvidia-smi
# !pip install prefetch_generator
# !pip install timm
# !pip install albumentations

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import trange
import torchvision
import boto3
import io
import sagemaker
from sagemaker import get_execution_role
import tempfile
import PIL
import time
from datetime import timedelta
from prefetch_generator import BackgroundGenerator
import timm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn import model_selection, metrics

In [3]:
# Make sure you're using cuda (GPU) by checking the hardware accelerator under Runtime -> Change runtime type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


# Download dataset from AWS S3

In [4]:
dataset_name = "Dataset"
if not os.path.exists(dataset_name):
    !pip install cloudpathlib[s3,gs,azure]
    from cloudpathlib import CloudPath
    cp = CloudPath("s3://cassavaproject")
    cp.download_to("./" + dataset_name)
else:
    print(f"File exists: {dataset_name}")

File exists: Dataset


# Preprocessing

In [5]:
class MyDataset(Dataset):

    def __init__(self, df, transform=None):
        self.image_path = 'Dataset/train_images'
        self.labels = df
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.image_path + '/' + self.labels.iloc[idx]['image_id']

        # Read the image from the file path
        #image = Image.open(img_name)
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = torchvision.io.read_file(img_name)
        #image = torchvision.io.decode_jpeg(image, device="cpu")
        #image = image.float().
        
        # Transform the image using self.transform
        if self.transform:
            image = self.transform(image=image)["image"]

        if "label" in self.labels.columns:
            label = self.labels.iloc[idx]['label']
            sample = (image, label)
        else:
            sample = (image)
        return sample

In [6]:
#Used for directly download and read file from AWS S3.
#If running the download cell above, use 'MyDataset' class instead of this one.
class MyDatasetS3(Dataset):

    def __init__(self, df, transform=None):
        #File path for csv and images
        self.image_path = 'train_images'
        
        #Connect to s3 file
        self.csv_path = df
        self.s3_client = boto3.resource('s3')
        self.bucket = self.s3_client.Bucket('cassavaproject')
        
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket = 'cassavaproject',Key = 'train.csv')

        self.labels = pd.read_csv(obj['Body'])
        
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.image_path + '/' + self.labels.iloc[idx]['image_id']
        
        # we can download the file from S3 to a temporary file locally, then store that opened file as our image variable.
        # we need to create the local file name
        obj = self.bucket.Object(img_name)
        tmp = tempfile.NamedTemporaryFile()
        tmp_name = '{}.jpg'.format(tmp.name)

        # now we can actually download from S3 to a local place
        with open(tmp_name, 'wb') as f:
            obj.download_fileobj(f)
            f.flush()
            f.close()
            
            image = torchvision.io.read_file(tmp_name)
            image = torchvision.io.decode_jpeg(image, device="cpu")
            image = image.float()
            
        # Transform the image using self.transform
        if self.transform:
            image = self.transform(image=image)["image"]

        if "label" in self.labels.columns:
            label = self.labels.iloc[idx]['label']
            sample = (image, label)
        else:
            sample = (image)
        return sample

In [7]:
# Credit: https://www.kaggle.com/code/aliabdin1/calculate-mean-std-of-images/notebook
mean = np.array([0.42984136, 0.49624753, 0.3129598 ])
std = np.array([0.21417203, 0.21910103, 0.19542212])

In [8]:
train_transform = torch.nn.Sequential(transforms.Resize((256,256),antialias=True), 
                                      transforms.Normalize(mean=mean,std=std))

In [9]:
albu_transform_train = A.Compose(
        [A.CenterCrop(height=256, width=256), 
         A.HorizontalFlip(p=0.5),
         A.VerticalFlip(p=0.5),
         A.RandomRotate90(p=0.5),
         A.RandomBrightnessContrast(p=0.8), 
         A.CoarseDropout(p=0.5),
         A.Normalize(mean=mean, std=std),
         ToTensorV2()
        ])

In [10]:
albu_transform_test = A.Compose(
        [A.CenterCrop(height=256, width=256),
         A.Normalize(mean=mean, std=std),
         ToTensorV2()
        ])

In [11]:
df = pd.read_csv('Dataset/train.csv')

train_df, test_df = model_selection.train_test_split(
    df, test_size=0.3, random_state=42, stratify=df.label.values
)

train_data = MyDataset(train_df, transform = albu_transform_train)
test_data = MyDataset(test_df, transform = albu_transform_test)

# Define our model

In [12]:
# timm.list_models('*resnet*')

In [13]:
test_model = timm.create_model('resnet34d', pretrained=True)
test_model.default_cfg

{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet34d_ra2-f8dcfcaf.pth',
 'num_classes': 1000,
 'input_size': (3, 224, 224),
 'pool_size': (7, 7),
 'crop_pct': 0.875,
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'first_conv': 'conv1.0',
 'classifier': 'fc',
 'architecture': 'resnet34d'}

In [14]:
class OurModel(nn.Module):
    def __init__(self, num_classes):
        super(OurModel, self).__init__()
        self.model = timm.create_model('resnet34d', pretrained=True) #torchvision.models.resnet34(weights='IMAGENET1K_V1')
        self.drop = nn.Dropout(p=0.2)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.softmax = nn.Softmax()
    def forward(self, input):
        out = self.model(input)
        out = self.drop(out)
        softmax = self.softmax(out)
        return softmax

# Train our model

In [15]:
model = OurModel(5).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=2e-5, weight_decay=1e-6, 
                             amsgrad=False) #torch.optim.Adam(model.parameters(), lr=5*1e-3)


In [16]:
class DataLoaderX(DataLoader):

    def __iter__(self):
        return BackgroundGenerator(super().__iter__())

In [17]:
train_loader = DataLoaderX(dataset=train_data, batch_size = 64, shuffle= True, num_workers=2, pin_memory=True)
test_loader = DataLoaderX(dataset=test_data, batch_size = 64, shuffle=False, num_workers=2, pin_memory=True)

In [18]:
print('Start fine-tuning...')

Start fine-tuning...


In [19]:
def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(images)

            _,prediction = torch.max(outputs.data, 1)
            correct += (prediction == labels).sum().item()
            total += labels.size(0)
        model.train()
        return 100 * correct / total

In [20]:
def get_run_time(start_time):
    end_time = time.time()
    runtime = end_time - start_time
    return runtime

In [ ]:
best_acc = 0.
best_epoch = None
end_patient = 0
num_epochs = 50
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=3, verbose=True)

save_model_path = ''
train_loss = []
train_accuracy = []
test_accuracy = []

start_time = time.time()


for epoch in trange(num_epochs):
    correct = 0
    total = 0
    epoch_loss = 0.
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss
        _, prediction = torch.max(outputs.data, 1)
        correct += (prediction == labels).sum().item()
        total += labels.size(0)

        time_diff = get_run_time(start_time)

        # print('Epoch [{}/{}]: Iter {}, Loss {:.4f}, Runtime {:.0f}m {:.0f}s'.format(epoch + 1, num_epochs, i + 1, loss, time_diff//60, time_diff%60))
        train_loss.append(loss)

    train_acc = 100 * correct / total
    print('Testing on test dataset...')
    test_acc = test_model(model, test_loader)
    print('Epoch [{}/{}] Loss: {:.4f} Train_Acc: {:.4f}  Test_Acc: {:.4f}'
          .format(epoch + 1, num_epochs, epoch_loss, train_acc, test_acc))
    scheduler.step(test_acc)
    train_accuracy.append(train_acc)
    test_accuracy.append(test_acc)
    if (test_acc > best_acc) & (test_acc >82):
        best_acc = test_acc
        best_epoch = epoch + 1
        print('The accuracy is improved, save model')
        torch.save(model.state_dict(), os.path.join(
                                                    'resnet34_acc_%g.pth' %
                                                    (best_acc)))

print('After the training, the end of the epoch %d, the accuracy %g is the highest' % (best_epoch, best_acc))

  0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipykernel_17912/3494929337.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax = self.softmax(out)


Testing on test dataset...


  2%|▏         | 1/50 [02:05<1:42:08, 125.07s/it]

Epoch [1/50] Loss: 337.7293 Train_Acc: 51.4055  Test_Acc: 67.8660
Testing on test dataset...


  4%|▍         | 2/50 [04:04<1:37:16, 121.59s/it]

...

Epoch [47/50] Loss: 253.3690 Train_Acc: 80.3833  Test_Acc: 83.7383
Epoch 00047: reducing learning rate of group 0 to 6.4000e-09.
Testing on test dataset...


 96%|█████████▌| 48/50 [1:30:47<02:54, 87.18s/it]

Epoch [48/50] Loss: 254.1409 Train_Acc: 80.1696  Test_Acc: 84.1277
Testing on test dataset...


 98%|█████████▊| 49/50 [1:32:09<01:25, 85.48s/it]

Epoch [49/50] Loss: 253.3171 Train_Acc: 80.0294  Test_Acc: 84.1277


# Result Analysis

In [ ]:
import matplotlib.pyplot as plt
import itertools

In [ ]:
plt.plot(np.arange(1,len(train_accuracy)+1),train_accuracy,label='Train Accuracy')
plt.plot(np.arange(1,len(test_accuracy)+1),test_accuracy,label='Test Accuracy')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Resnet34 Accuracy Versus Epochs')
plt.show()

In [ ]:
nb_classes = 5

confusion_matrix = torch.zeros(nb_classes, nb_classes)
model.eval()
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        #print(classes)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm/ np.sum(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2%' #if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure()
num_classes=5
M = confusion_matrix.numpy()
plot_confusion_matrix(M, classes=np.arange(num_classes), 
                      normalize=True,title="Resnet34 Confusion Matrix")
print("Accuracy:",np.trace(M/np.sum(M)))

###### 